<a href="https://colab.research.google.com/github/Arseniy-Polyakov/applied_linguistics_course/blob/main/Task_6_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Устанавливаем библиотеку Natasha для извлечения именованных сущностей

In [42]:
!pip install natasha json_repair

Подключаем модули, необходимые для предобработки текста, а также извлечения и нормализации именованных сущностей

In [43]:
import re
import folium
import pandas as pd
import json_repair
from google.colab import userdata
from huggingface_hub import InferenceClient
from natasha import Segmenter, Doc, NewsNERTagger, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, MorphVocab

Загружаем корпус советских песен

In [3]:
df = pd.read_excel("corpus.xlsx")
len(df)

1573

Определим количество униклаьных меток, по которым будем выбирать подкорпус

In [4]:
tags = [re.split(r", |\n", tag) for tag in list(df["Tag"])]
tags_unique = set([re.sub(r"[, ]", "", word) for tag in tags for word in tag if word])
tags_unique

{'вера',
 'война',
 'герои',
 'город',
 'грусть',
 'детство',
 'дом',
 'другое',
 'дружба',
 'жизнь',
 'космос',
 'любовь',
 'люди',
 'мама',
 'мать',
 'надежда',
 'одиночество',
 'память',
 'победа',
 'подвиг',
 'природа',
 'путешествие',
 'радость',
 'разлука',
 'революция',
 'религия',
 'родина',
 'родины',
 'свобода',
 'сказка',
 'смерть',
 'спорт',
 'судьба',
 'творчество',
 'труд'}

В качестве исследования сформируем следующую гипотезу: в рамках тегов "война", "герои", "память", "победа", "подвиг" (военный кластер) будут чаще встречаться города-герои, города воинской славы и трудовой доблести; для кластера "дом" с тегами "детство", "дом", "мать", "мама", "родина" будут больше характерны топонимы деревень, сел; топонимы, не относящиеся к центральной части России (СССР)

Сделаем на основе двух выделенных кластеров два подкорпуса: "война" и "дом"

In [5]:
corpus = df["lyrics_text"]
tags = df["Tag"]
lyrics_tags = {}
for i in range(len(corpus)):
  lyrics_tags[tags[i]] = corpus[i]
war_tags = ["война", "герои", "память", "победа", "подвиг"]
home_tags = ["детство", "дом", "мать", "мама", "родина"]
war_corpus = []
home_corpus = []
for tag in war_tags:
  for song in list(lyrics_tags.keys()):
    if tag in song:
      war_corpus.append(lyrics_tags[song])
for tag in home_tags:
  for song in list(lyrics_tags.keys()):
    if tag in song:
      home_corpus.append(lyrics_tags[song])

war_corpus_preprocessed = re.sub(r"\n", " ", " ".join(war_corpus))
home_corpus_preprocessed = re.sub(r"\n", " ", " ".join(home_corpus))

Определяем необходимые зависимости для задач NER и нормализации (сегментацию для NER; сегментацию, частеречную разметку и синтаксический парсинг для нормализации именованных сущностей)

In [6]:
segmenter_war = Segmenter()
emb_war = NewsEmbedding()
morph_vocab_war = MorphVocab()
ner_tagger_war = NewsNERTagger(emb_war)
morph_tagger_war = NewsMorphTagger(emb_war)
syntax_parser_war = NewsSyntaxParser(emb_war)

Произведем извлечение именованных сущностей для военного кластера (с учетом нормализации)

In [7]:
doc = Doc(war_corpus_preprocessed)
doc.segment(segmenter_war)
doc.tag_morph(morph_tagger_war)
doc.parse_syntax(syntax_parser_war)
doc.tag_ner(ner_tagger_war)

Найдем частотное распределение топонимов, связанных тегами с военным кластером. В финальную выборку определим только те тономимы, частотность встречаемости которых больше 10 (для более наглядной визуализации)



In [8]:
war_locations_normalized = [span.normalize(morph_vocab_war) for span in doc.spans]
war_locations_normalized = [span.normal for span in doc.spans if span.type == "LOC"]
war_locations_dict = {}
for location in war_locations_normalized:
  if location not in war_locations_dict:
    war_locations_dict[location] = 1
  else:
    war_locations_dict[location] += 1
for location in list(war_locations_dict.items()):
  if war_locations_dict[location[0]] < 3:
    del war_locations_dict[location[0]]
war_locations_dict = dict(sorted(war_locations_dict.items(), key = lambda x:x[1], reverse=True))
war_locations_dict

{'Россия': 85,
 'Москва': 78,
 'Земля': 63,
 'Гренада': 32,
 'Варшава': 28,
 'Берлин': 20,
 'Вспомним': 20,
 'Париж': 18,
 'Питерская': 18,
 'Русь': 17,
 'Сидящие': 15,
 'Волга': 14,
 'Нева': 14,
 'Питерская Пронесся': 14,
 'Грохот': 12,
 'Афганистан': 11,
 'Фламенго': 11,
 'Марьина роща': 11,
 'Садовая': 10,
 'Скамеечка кленовая': 10,
 'Ростов-город': 10,
 'Болгария': 10,
 'Херсон': 10,
 'Манжерок': 10,
 'Нарвой': 9,
 'Одесса': 8,
 'Кронштадт': 8,
 'Бухенвальд': 8,
 'Дон': 8,
 'Украина': 7,
 'Севастополь': 7,
 'Родина': 7,
 'Ленинград': 7,
 'О': 7,
 'Крым': 7,
 'Черное море': 7,
 'Курск': 6,
 'Братск': 6,
 'Калуга': 6,
 'Иртыш': 6,
 'Звенигород': 6,
 'Ходившие': 5,
 'Брест': 5,
 'Орел': 5,
 'Красная площадь': 5,
 'Гремящий': 5,
 'Урал': 5,
 'Бреста': 5,
 'Невский': 5,
 'Финский залив': 5,
 'Солнце': 5,
 'Смоленская дорога': 5,
 'Америка': 5,
 'Ростов': 4,
 'Запад': 4,
 'Смоленск': 4,
 'Тула': 4,
 'Восток': 4,
 'Брянская улица': 4,
 'Минская улица': 4,
 'Минск': 4,
 'Брестская улица': 

Произведем извлечение именнованных сущностей для кластера "дом"

In [9]:
segmenter_home = Segmenter()
emb_home = NewsEmbedding()
morph_vocab_home = MorphVocab()
ner_tagger_home = NewsNERTagger(emb_home)
morph_tagger_home = NewsMorphTagger(emb_home)
syntax_parser_home = NewsSyntaxParser(emb_home)

In [10]:
doc = Doc(home_corpus_preprocessed)
doc.segment(segmenter_home)
doc.tag_morph(morph_tagger_home)
doc.parse_syntax(syntax_parser_home)
doc.tag_ner(ner_tagger_home)

Найдем частотное распределение топонимов, связанных тегами с кластером дома. В финальную выборку определим только те тономимы, частотность встречаемости которых больше 4 (для более наглядной визуализации)




In [11]:
home_locations_normalized = [span.normalize(morph_vocab_home) for span in doc.spans]
home_locations_normalized = [span.normal for span in doc.spans if span.type == "LOC"]
home_locations_dict = {}
for location in home_locations_normalized:
  if location not in home_locations_dict:
    home_locations_dict[location] = 1
  else:
    home_locations_dict[location] += 1
for location in list(home_locations_dict.items()):
  if home_locations_dict[location[0]] < 3:
    del home_locations_dict[location[0]]
home_locations_dict = dict(sorted(home_locations_dict.items(), key = lambda x:x[1], reverse=True))
home_locations_dict

{'Россия': 64,
 'Москва': 41,
 'Земля': 37,
 'Волга': 12,
 'Гренада': 12,
 'Нева': 11,
 'Марьина роща': 11,
 'Советский Союз': 10,
 'Одесса': 9,
 'Америка': 9,
 'Горький': 8,
 'Кремль': 7,
 'Африка': 6,
 'Солнечный луч': 6,
 'Сибирь': 5,
 'Париж': 5,
 'Арбат': 4,
 'Свердлов': 4,
 'За Охотный ряд': 4,
 'На Земля': 4,
 'Баку': 4,
 'Берлин': 4,
 'Подмосковная': 4,
 'Горное': 4,
 'Керчь': 4,
 'Русь': 4,
 'Аляска': 4,
 'Бухенвальд': 4,
 'Камчатка О-о': 4,
 'Украина': 3,
 'Ростов': 3,
 'Иран': 3,
 'Ленинград': 3,
 'Кронштадт': 3,
 'Пекин': 3,
 'Нам': 3,
 'Братск': 3,
 'Тусе': 3,
 'О': 3,
 'Металлурги': 3,
 'Испания': 3,
 'Степь': 3,
 'Ленинские горы': 3,
 'Калуга': 3,
 'Марьина': 3}

Визуализируем самые часто встречающиеся города среди кластера "война" на карте. Координаты были найдены на [сайте](https://time-in.ru/coordinates), однако потенциально поиск координат может быть автоматизирован с помощью API Яндекс Карт (является платным источником)

In [14]:
war_cities = pd.DataFrame({"city": ["Москва", "Одесса", "Горький", "Париж", "Баку", "Берлин", "Керчь", "Ростов",
                                    "Ленинград", "Кронштадт", "Пекин", "Братск", "Калуга"],
                          "frequency": [41, 9, 8, 5, 4, 4, 4, 3, 3, 3, 3, 3, 3]})

coordinates = {"Москва": (55.7558, 37.6176),
               "Одесса": (46.4775, 30.7326),
               "Горький": (56.3287, 44.002),
               "Париж": (48.8534, 2.3488),
               "Баку": (40.3777, 49.892),
               "Берлин": (52.5244, 13.4105),
               "Керчь": (45.3531, 36.4743),
               "Ростов": (47.2313, 39.7233),
               "Ленинград": (59.9386, 30.3141),
               "Кронштадт": (59.5943, 29.4600),
               "Пекин": (39.9075, 116.397),
               "Братск": (39.9075, 116.397),
               "Калуга": (54.5293, 36.2754)}

map = folium.Map(location=[55.7558, 37.6176], zoom_start=4)

for index, row in war_cities.iterrows():
    city = row['city']
    frequency = row['frequency']
    lat, lon = coordinates[city]
    folium.CircleMarker([lat, lon], radius=frequency/10, popup=city).add_to(map)

map.save("war_cities.html")

Визуализируем самые часто встречающиеся города среди кластера "дом" на карте


In [15]:
home_cities = pd.DataFrame({"city": ["Москва", "Одесса", "Горький", "Париж", "Баку", "Берлин", "Керчь", "Ростов",
                                    "Ленинград", "Кронштадт", "Пекин", "Братск", "Калуга"],
                          "frequency": [41, 9, 8, 5, 4, 4, 4, 3, 3, 3, 3, 3, 3]})

coordinates = {"Москва": (55.7558, 37.6176),
               "Одесса": (46.4775, 30.7326),
               "Горький": (56.3287, 44.002),
               "Париж": (48.8534, 2.3488),
               "Баку": (40.3777, 49.892),
               "Берлин": (52.5244, 13.4105),
               "Керчь": (45.3531, 36.4743),
               "Ростов": (47.2313, 39.7233),
               "Ленинград": (59.9386, 30.3141),
               "Кронштадт": (59.5943, 29.4600),
               "Пекин": (39.9075, 116.397),
               "Братск": (39.9075, 116.397),
               "Калуга": (54.5293, 36.2754)}

map = folium.Map(location=[55.7558, 37.6176], zoom_start=4)

for index, row in home_cities.iterrows():
    city = row['city']
    frequency = row['frequency']
    lat, lon = coordinates[city]
    folium.CircleMarker([lat, lon], radius=frequency/10, popup=city).add_to(map)

map.save("home_cities.html")

Вывод: гипотеза касаемо военного кластера частично подтвердилась, действительно в песнях часто упомянаются города-герои, а также города, в которых происходили ожесточенные бои (преимущественно в период Великой Отечественной войны), а именно города: Москва, Одесса, Нижний Новгород (Горький), Керчь и другие.

Однако некоторые города, в которых не было боев в период Великой Отечественной войны также попали в данную выборку, а именно города Пекин, Братск. Есть гипотеза (не изучая лирику), что Братск в песнях упоминается в значении трудового города, города стройки Байкало-Амурской магистрали, поэтому воспевается как город героев, труженников-железнодорожников, как подвиг народов СССР. Насчет Пекина есть гипотеза о том, что он воспевается как город, в котором были сражения в контексте Второй мировой войны, сражения против Японии.


Также в результате анализа было выяснено, что два кластера "война" и "дом" являются взаимозаменяющими на данном датасете, результаты по частотным городам совпадают (общее количество сущностей разное, как и частотность употребления сущностей, не связанных с наименованием городов), поэтому в данном случае заявленную ранее гипотезу нельзя ни доказать, ни опровергнуть


Проведем сравнение библиотеки natasha с большими языковыми моделями (meta-llama/Llama-4-Scout-17B-16E-Instruct) для извлечения именнованных сущностей локаций (тэг LOC) для кластеров "война" и "дом"

Выполним подключение к модели meta-llama/Llama-4-Scout-17B-16E-Instruct провайдера novita с помощью Hugging Face API

In [80]:
def llm_ner(model: str, provider: str, text: str) -> str:
  client = InferenceClient(
    provider=provider,
    api_key=userdata.get("API_KEY")
  )

  completion = client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "system",
            "content": f"""You are an expert in finding Named Entities in the text.
            Your goal is to find all location names (all geographical objets: countries, cities, regions)
            in the text {text} and count how many times they used in the text.
            You must provide an answer in a JSON format. Sort the final JSON from the top to the bottom.
            For example:
            'Москва': 15,
            'Санкт-Петербург': 10,
            'Казань': 8,
            'Нижний Новгород': 5,
            'Пермь': 3
            You must strictly provide an answer only in Russian. Do not add any additional information just provide a JSON sctructure.
            Do not add any additional cities which you know, just add locations which you found in the text
            """
        }
    ],
  )
  return completion.choices[0].message

Выведем ответ модели и преобразуем к json формату

In [ ]:
model = "meta-llama/Llama-4-Scout-17B-16E-Instruct"
provider = "novita"
llm_answer_war = llm_ner(model, provider, war_corpus_preprocessed).content
llm_answer_war_json = json_repair.loads(llm_answer_war)

Многие большие языковые модели плохо справляются с подсчетом количественных метрик, велик риск галлюцинаций. В связи с этим более релевантной выглядит задача извлечения сущностей без подсчета частоты их встречаемости в тексте.

Посчитаем количество топонимов, которые встретились среди выборки, сделанной с помощью natasha, срели выборки, сделанной с помощью модели meta-llama/Llama-4-Scout-17B-16E-Instruct. Найдем общие топонимы

In [75]:
llama_entities_war = [re.sub(r"[^А-Яа-яёЁ\s]", "", item) for item in list(llm_answer_war_json.keys())]
natasha_entities_war = list(war_locations_dict.keys())
count_same = len([item for item in llama_entities_war if item in natasha_entities_war])
print(f"Количество топонимов, которые встретились и в выборке llama и в выборке natasha {count_same}")
print(f"Общее количество топонимов, которые есть в выборке natasha {len(natasha_entities_war)}")
print(f"Общее количество топонимов, которые есть в выборке llama {len(llama_entities_war)}")

Количество топонимов, которые встретились и в выборке llama и в выборке natasha 39
Общее количество топонимов, которые есть в выборке natasha 121
Общее количество топонимов, которые есть в выборке llama 117


Вывод: большая языковая модель meta-llama/Llama-4-Scout-17B-16E-Instruct не лучшим образом справляется с извлечением именованных сущностей на данных корпуса советской песни. В данном конкретном кейсе стоит использовать библиотеку natasha, которая даст большую точность